In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!head -n 10 "/content/drive/MyDrive/Colab Notebooks/BERT Research/clean_id.txt"

head: cannot open '/content/drive/MyDrive/Colab Notebooks/BERT Research/clean_id.txt' for reading: No such file or directory


In [4]:
! pip install torch==2.1.0
! pip install tokenizers==2.4.1
! pip install transformers==2.11.0

ERROR: Could not find a version that satisfies the requirement tokenizers==2.4.1 (from versions: 0.0.2, 0.0.3, 0.0.4, 0.0.5, 0.0.6, 0.0.7, 0.0.8, 0.0.9, 0.0.10, 0.0.11, 0.0.12, 0.0.13, 0.1.0, 0.1.1, 0.2.0, 0.2.1, 0.3.0, 0.4.0, 0.4.1, 0.4.2, 0.5.0, 0.5.1, 0.5.2, 0.6.0, 0.7.0rc1, 0.7.0rc2, 0.7.0rc3, 0.7.0rc4, 0.7.0rc5, 0.7.0rc6, 0.7.0rc7, 0.7.0, 0.8.0.dev0, 0.8.0.dev1, 0.8.0.dev2, 0.8.0rc1, 0.8.0rc2, 0.8.0rc3, 0.8.0rc4, 0.8.0, 0.8.1rc1, 0.8.1rc2, 0.8.1, 0.9.0.dev0, 0.9.0.dev1, 0.9.0.dev2, 0.9.0.dev3, 0.9.0.dev4, 0.9.0rc1, 0.9.0rc2, 0.9.0, 0.9.1, 0.9.2, 0.9.3, 0.9.4, 0.10.0rc1, 0.10.0, 0.10.1rc1, 0.10.1, 0.10.2, 0.10.3, 0.11.0, 0.11.1, 0.11.2, 0.11.3, 0.11.4, 0.11.5, 0.11.6, 0.12.0, 0.12.1, 0.13.0, 0.13.1, 0.13.2, 0.13.3rc1, 0.13.3, 0.13.4rc1, 0.13.4rc2, 0.13.4rc3, 0.14.0rc1, 0.14.0, 0.14.1rc1, 0.14.1, 0.15.0rc1, 0.15.0)
ERROR: No matching distribution found for tokenizers==2.4.1
  Using cached transformers-2.11.0-py3-none-any.whl (674 kB)
  Using cached tokenizers-0.7.0.tar.gz (81 kB)
  

In [ ]:
!CUDA_LAUNCH_BLOCKING=1

In [ ]:
import tokenizers

In [ ]:
bwpt = tokenizers.BertWordPieceTokenizer()

paths = ["/content/drive/MyDrive/Colab Notebooks/BERT Research/clean_id_100k.txt"]

bwpt.train(
    files=paths,
    vocab_size=50000,
    min_frequency=3,
    limit_alphabet=1000
)

bwpt.save('.', 'daily-id')

['./daily-id-vocab.txt']

In [ ]:
from transformers import BertTokenizer, LineByLineTextDataset

In [ ]:
vocab_file_dir = "./daily-id-vocab.txt"
tokenizer = BertTokenizer.from_pretrained(vocab_file_dir)

Calling BertTokenizer.from_pretrained() with the path to a single file or url is deprecated


In [ ]:
sentence = "saya mau permakanan nasi"
enc = tokenizer.tokenize(sentence)
enc

['saya', 'mau', 'permak', '##anan', 'nasi']

In [ ]:
dataset= LineByLineTextDataset(
    tokenizer = tokenizer,
    file_path = '/content/drive/MyDrive/Colab Notebooks/BERT Research/clean_id_100k.txt',
    block_size = 128  # maximum sequence length
)

print('No. of lines: ', len(dataset)) # No of lines in your datset

No. of lines:  100000


## Defining Model

In [ ]:
from transformers import BertConfig, BertForMaskedLM, DataCollatorForLanguageModeling

config = BertConfig(
    vocab_size=50000,
    hidden_size=768,
    num_hidden_layers=6,
    num_attention_heads=12,
    max_position_embeddings=512
)

model = BertForMaskedLM(config)
print('No of parameters: ', model.num_parameters())

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=True, mlm_probability=0.15
)

No of parameters:  82556240


## Defining Training Arguments

In [ ]:
from transformers import Trainer, TrainingArguments

In [ ]:
training_args = TrainingArguments(
    output_dir='./models',
    overwrite_output_dir=True,
    num_train_epochs=4,
    per_device_train_batch_size=64,
    save_steps=1000,
    save_total_limit=2,
)

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=dataset,
    prediction_loss_only=True,
)

In [ ]:
%%time
trainer.train()
trainer.save_model('./models')

Epoch:   0%|          | 0/4 [00:00<?, ?it/s]

Iteration:   0%|          | 0/1563 [00:00<?, ?it/s]

{"loss": 7.760995920181275, "learning_rate": 4.6001279590531034e-05, "epoch": 0.3198976327575176, "step": 500}
{"loss": 7.06457195186615, "learning_rate": 4.2002559181062066e-05, "epoch": 0.6397952655150352, "step": 1000}


/usr/local/lib/python3.7/dist-packages/torch/optim/lr_scheduler.py:200: UserWarning: Please also save or load the state of the optimzer when saving or loading the scheduler.
  warnings.warn(SAVE_STATE_WARNING, UserWarning)


{"loss": 6.8648159189224245, "learning_rate": 3.800383877159309e-05, "epoch": 0.9596928982725528, "step": 1500}


...

{"loss": 6.70471448135376, "learning_rate": 3.400511836212412e-05, "epoch": 1.2795905310300704, "step": 2000}
{"loss": 6.581752689361572, "learning_rate": 3.0006397952655153e-05, "epoch": 1.599488163787588, "step": 2500}
{"loss": 6.450511471748352, "learning_rate": 2.600767754318618e-05, "epoch": 1.9193857965451055, "step": 3000}


Iteration:   0%|          | 0/1563 [00:00<?, ?it/s]

{"loss": 6.3564983768463135, "learning_rate": 2.2008957133717213e-05, "epoch": 2.239283429302623, "step": 3500}
{"loss": 6.270851696014405, "learning_rate": 1.801023672424824e-05, "epoch": 2.5591810620601407, "step": 4000}
{"loss": 6.21679706287384, "learning_rate": 1.4011516314779271e-05, "epoch": 2.8790786948176583, "step": 4500}


Iteration:   0%|          | 0/1563 [00:00<?, ?it/s]

{"loss": 6.158131644248963, "learning_rate": 1.0012795905310301e-05, "epoch": 3.198976327575176, "step": 5000}
{"loss": 6.115304460525513, "learning_rate": 6.014075495841331e-06, "epoch": 3.5188739603326935, "step": 5500}
{"loss": 6.083669279098511, "learning_rate": 2.015355086372361e-06, "epoch": 3.838771593090211, "step": 6000}
CPU times: user 1h 27min 22s, sys: 29.3 s, total: 1h 27min 51s
Wall time: 1h 28min 6s


In [ ]:
from transformers import pipeline

In [ ]:
model = BertForMaskedLM.from_pretrained('./models')


In [ ]:
fill_mask = pipeline(
    "fill-mask",
    model=model,
    tokenizer=tokenizer
)

In [ ]:
fill_mask("apa [MASK] kamu mau?")

[{'score': 0.10761164128780365,
  'sequence': '[CLS] apa yang kamu mau? [SEP]',
  'token': 206},
 {'score': 0.1056077629327774,
  'sequence': '[CLS] apa, kamu mau? [SEP]',
  'token': 15},
 {'score': 0.028667578473687172,
  'sequence': '[CLS] apa tidak kamu mau? [SEP]',
  'token': 289},
 {'score': 0.026282286271452904,
  'sequence': '[CLS] apa kita kamu mau? [SEP]',
  'token': 432},
 {'score': 0.02084304392337799,
  'sequence': '[CLS] apa saya kamu mau? [SEP]',
  'token': 433}]

In [ ]:
fill_mask("apa [MASK] mereka berdua?")

[{'score': 0.12286432832479477,
  'sequence': '[CLS] apa, mereka berdua? [SEP]',
  'token': 15},
 {'score': 0.11828538030385971,
  'sequence': '[CLS] apa yang mereka berdua? [SEP]',
  'token': 206},
 {'score': 0.025175970047712326,
  'sequence': '[CLS] apa tidak mereka berdua? [SEP]',
  'token': 289},
 {'score': 0.01976119354367256,
  'sequence': '[CLS] apa kita mereka berdua? [SEP]',
  'token': 432},
 {'score': 0.01854696124792099,
  'sequence': '[CLS] apa ini mereka berdua? [SEP]',
  'token': 240}]

In [ ]:
fill_mask("aku memohon pada [MASK] untuk mengampuniku")

[{'score': 0.009659789502620697,
  'sequence': '[CLS] aku memohon pada hari untuk mengampuniku [SEP]',
  'token': 430},
 {'score': 0.007857121527194977,
  'sequence': '[CLS] aku memohon pada itu untuk mengampuniku [SEP]',
  'token': 254},
 {'score': 0.007484164088964462,
  'sequence': '[CLS] aku memohon pada ini untuk mengampuniku [SEP]',
  'token': 240},
 {'score': 0.0074275401420891285,
  'sequence': '[CLS] aku memohon pada terjadi untuk mengampuniku [SEP]',
  'token': 687},
 {'score': 0.006930225994437933,
  'sequence': '[CLS] aku memohon pada dilakukan untuk mengampuniku [SEP]',
  'token': 558}]

In [ ]:
! tar -czvf bert_news_id_100k.tar.gz /content/models /content/daily-id-vocab.txt

tar: Removing leading `/' from member names
/content/models/
/content/models/checkpoint-5000/
/content/models/checkpoint-5000/config.json
/content/models/checkpoint-5000/optimizer.pt
/content/models/checkpoint-5000/scheduler.pt
/content/models/checkpoint-5000/training_args.bin
/content/models/checkpoint-5000/pytorch_model.bin
/content/models/checkpoint-6000/
/content/models/checkpoint-6000/config.json
/content/models/checkpoint-6000/optimizer.pt
/content/models/checkpoint-6000/scheduler.pt
/content/models/checkpoint-6000/training_args.bin
/content/models/checkpoint-6000/pytorch_model.bin
/content/models/config.json
/content/models/training_args.bin
/content/models/pytorch_model.bin
/content/daily-id-vocab.txt


In [ ]:
! cp /content/bert_news_id_100k.tar.gz "/content/drive/MyDrive/Colab Notebooks/BERT Research/bert_news_id_100k.tar.gz"